In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
import xgboost
import datetime

In [3]:
data = pd.read_csv('./data/train.csv')

In [4]:
data.head()

,ct2010,pickup_date_time,nb_pickup
0,100,2015-01-01 01:00:00,70
1,100,2015-01-01 02:00:00,62
2,100,2015-01-01 03:00:00,34
3,100,2015-01-01 04:00:00,16
4,100,2015-01-01 05:00:00,5


In [5]:
data.pickup_date_time.value_counts()

2015-05-03 01:00:00    622
2015-05-03 02:00:00    612
2015-01-01 03:00:00    610
2015-01-01 04:00:00    599
2015-01-01 02:00:00    596
2015-01-01 01:00:00    575
2015-01-01 05:00:00    523
2015-03-08 03:00:00    504
2015-03-08 00:00:00    497
2015-05-03 03:00:00    497
2015-03-29 00:00:00    494
2015-05-02 23:00:00    493
2015-03-08 04:00:00    492
2015-03-01 01:00:00    491
2015-04-26 00:00:00    489
2015-03-08 01:00:00    485
2015-02-15 00:00:00    485
2015-02-15 01:00:00    483
2015-04-19 01:00:00    482
2015-05-02 22:00:00    482
2015-03-22 01:00:00    482
2015-04-12 01:00:00    481
2015-04-19 00:00:00    479
2015-02-28 00:00:00    475
2015-03-07 01:00:00    475
2015-04-26 01:00:00    474
2015-02-01 23:00:00    473
2015-04-12 00:00:00    473
2015-02-22 00:00:00    472
2015-05-17 01:00:00    469
                      ... 
2015-03-17 03:00:00    282
2015-02-09 03:00:00    282
2015-05-26 03:00:00    280
2015-01-27 08:00:00    280
2015-03-23 03:00:00    280
2015-01-21 03:00:00    278
2

In [6]:
data['pickup_date_time'] = pd.to_datetime(data['pickup_date_time'])

In [7]:
data.head()

,ct2010,pickup_date_time,nb_pickup
0,100,2015-01-01 01:00:00,70
1,100,2015-01-01 02:00:00,62
2,100,2015-01-01 03:00:00,34
3,100,2015-01-01 04:00:00,16
4,100,2015-01-01 05:00:00,5


In [8]:
def month(d):
    return d.month

data['month'] =  data['pickup_date_time'].apply(month)

def year(d):
    return d.year

data['year'] =  data['pickup_date_time'].apply(year)

def day(d):
    return d.day

data['day'] =  data['pickup_date_time'].apply(day)

In [9]:
data.head()

,ct2010,pickup_date_time,nb_pickup,month,year,day
0,100,2015-01-01 01:00:00,70,1,2015,1
1,100,2015-01-01 02:00:00,62,1,2015,1
2,100,2015-01-01 03:00:00,34,1,2015,1
3,100,2015-01-01 04:00:00,16,1,2015,1
4,100,2015-01-01 05:00:00,5,1,2015,1


In [10]:
month_1_avg = data[data['month'] == 1]['nb_pickup'].median()
month_2_avg = data[data['month'] == 2]['nb_pickup'].median()
month_3_avg = data[data['month'] == 3]['nb_pickup'].median()
month_4_avg = data[data['month'] == 4]['nb_pickup'].median()
month_5_avg = data[data['month'] == 5]['nb_pickup'].median()

In [11]:
data.head()

,ct2010,pickup_date_time,nb_pickup,month,year,day
0,100,2015-01-01 01:00:00,70,1,2015,1
1,100,2015-01-01 02:00:00,62,1,2015,1
2,100,2015-01-01 03:00:00,34,1,2015,1
3,100,2015-01-01 04:00:00,16,1,2015,1
4,100,2015-01-01 05:00:00,5,1,2015,1


In [12]:
def month_avg(m):
    if (m == 1):
        return month_1_avg
    if (m == 2):
        return month_2_avg
    if (m == 3):
        return month_3_avg
    if (m == 4):
        return month_4_avg
    if (m == 5):
        return month_5_avg
    
    
data['month_avg'] =  data['month'].apply(month_avg)

In [13]:
data.head()

,ct2010,pickup_date_time,nb_pickup,month,year,day,month_avg
0,100,2015-01-01 01:00:00,70,1,2015,1,9.0
1,100,2015-01-01 02:00:00,62,1,2015,1,9.0
2,100,2015-01-01 03:00:00,34,1,2015,1,9.0
3,100,2015-01-01 04:00:00,16,1,2015,1,9.0
4,100,2015-01-01 05:00:00,5,1,2015,1,9.0


In [14]:
data.month_avg.value_counts()

10.0    785080
9.0     493674
Name: month_avg, dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278754 entries, 0 to 1278753
Data columns (total 3 columns):
ct2010              1278754 non-null int64
pickup_date_time    1278754 non-null datetime64[ns]
nb_pickup           1278754 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 29.3 MB


In [10]:
def week_day(d):
    return d.weekday()
    
data['weekday'] = data['pickup_date_time'].apply(week_day)

In [11]:
data = data.drop('pickup_date_time', axis=1)

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [13]:
target = data['nb_pickup']
features = scaler.fit_transform(data.drop('nb_pickup', axis=1))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                   test_size = 0.2,
                                                   random_state = 0)

In [30]:
# model = DecisionTreeRegressor()
# model = xgboost.XGBRegressor(n_estimators=100, max_depth=7)
model = BaggingRegressor(RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4), n_estimators=100, n_jobs=-1, bootstrap=True)

In [31]:
# dtmodel.fit(X_train, y_train)
model.fit(X_train, y_train)
#xgb.fit(X_train, y_train)
#dtmodel.fit(X_train, y_train)

BaggingRegressor(base_estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=100, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [32]:
submission_data = pd.read_csv('./data/submission_sample.csv')

In [33]:
def break_key(key):
    return int(key.split('/')[0])

def break_date(key):
    return key.split('/')[1]

submission_data['ct2010'] = submission_data['key'].apply(break_key)
submission_data['pickup_date_time'] = submission_data['key'].apply(break_date)
submission_data['pickup_date_time'] = pd.to_datetime(submission_data['pickup_date_time'])

In [36]:
submission_data['month'] =  submission_data['pickup_date_time'].apply(month)

In [41]:
submission_data['pickup_date_time']

0        2015-06-01 01:00:00
1        2015-06-01 02:00:00
2        2015-06-01 03:00:00
3        2015-06-01 04:00:00
4        2015-06-01 05:00:00
5        2015-06-01 06:00:00
6        2015-06-01 07:00:00
7        2015-06-01 08:00:00
8        2015-06-01 09:00:00
9        2015-06-01 10:00:00
10       2015-06-01 11:00:00
11       2015-06-01 12:00:00
12       2015-06-01 13:00:00
13       2015-06-01 14:00:00
14       2015-06-01 15:00:00
15       2015-06-01 16:00:00
16       2015-06-01 17:00:00
17       2015-06-01 18:00:00
18       2015-06-01 19:00:00
19       2015-06-01 20:00:00
20       2015-06-01 21:00:00
21       2015-06-01 22:00:00
22       2015-06-01 23:00:00
23       2015-06-02 00:00:00
24       2015-06-02 01:00:00
25       2015-06-02 02:00:00
26       2015-06-02 03:00:00
27       2015-06-02 04:00:00
28       2015-06-02 05:00:00
29       2015-06-02 06:00:00
                 ...        
250487   2015-06-09 14:00:00
250488   2015-06-09 19:00:00
250489   2015-06-22 21:00:00
250490   2015-

In [34]:
# submission_data['year'] =  submission_data['pickup_date_time'].apply(year)
# submission_data['day'] =  submission_data['pickup_date_time'].apply(day)
submission_data['weekday'] = submission_data['pickup_date_time'].apply(week_day)
# submission_data['month_avg'] = submission_data['month'].apply(month_avg)

In [35]:
submission_data = submission_data.drop('key', axis=1)

In [36]:
submission_data = submission_data.drop('nb_pickup', axis=1)

In [22]:
submission_data

# submission_data['date_delta'] = (submission_data['pickup_date_time'] - submission_data['pickup_date_time'].min())  / np.timedelta64(1,'D')

,ct2010,pickup_date_time,weekday
0,100,2015-06-01 01:00:00,0
1,100,2015-06-01 02:00:00,0
2,100,2015-06-01 03:00:00,0
3,100,2015-06-01 04:00:00,0
4,100,2015-06-01 05:00:00,0
5,100,2015-06-01 06:00:00,0
6,100,2015-06-01 07:00:00,0
7,100,2015-06-01 08:00:00,0
8,100,2015-06-01 09:00:00,0
9,100,2015-06-01 10:00:00,0


In [37]:
submission_data = submission_data.drop('pickup_date_time', axis=1)

In [38]:
submission_data = scaler.fit_transform(submission_data)

In [39]:
# submission_data_predictions1 = dtmodel.predict(submission_data)
submission_data_predictions = model.predict(submission_data)
#submission_data_predictions = xgb.predict(submission_data)

In [40]:
np.round(submission_data_predictions)

array([30., 30., 30., ...,  1.,  1.,  1.])

In [41]:
new_submission_data = pd.read_csv('./data/submission_sample.csv')

In [42]:
new_submission_data['nb_pickup'] = np.round(submission_data_predictions)

In [44]:
new_submission_data.to_csv('finel_final_submit.csv', index=False)